In [ ]:
import pandas as pd  
import numpy as np  
from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import classification_report, confusion_matrix  
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline
import warnings
#warnings.filterwarnings('ignore')


In [ ]:
#df = pd.read_csv('C:\\Users\\fd299212\\Desktop\\lab_Stuff\\collaborations\\cady\\machineLearning\\lyme_data_gcfp.txt',sep='\t')
df = pd.read_csv('C:\\Users\\fd299212\\lyme_data_20220520.csv')
df.head()

In [ ]:
#create a new column in DF and fully populate with "Neg"
#then alter to Pos for any 'Diag' column values that are not equal to "Neg" (various positive states)
df['bin_diag'] = "Neg"
df.loc[df['Diag']!="Neg", 'bin_diag'] = "Pos"


In [ ]:
df.head()

In [ ]:
#df['Diag'].value_counts()
df.info()

In [ ]:
df.columns
#df.dtypes

In [ ]:
from sklearn.model_selection import train_test_split
#X = df.drop(['Diag','ID'], axis=1)  
#X = df.filter(['VlsE', 'DbpA', 'P58', 'OspC','ErpL','DbpB'],axis=1)  
X = df.filter(['VlsE', 'DbpA', 'P58', 'OspC','ErpL','P66'],axis=1)  

#Data Standardization gives the data zero mean and unit variance, it is considered good practice, 
#especially for algorithms such as KNN which is based on the distance of data points
#however, there is some disagreement about it for logistic regression...may require testing for specific dataset results
X = preprocessing.StandardScaler().fit(X).transform(X.astype(float))
X[0:5]
y = df['bin_diag']
#split original dataset into training and testing subsets
#stratify=y ensures that the sampled sets attempt to represent each class's proportions as they were in the full set
#the 'y' does not mean 'yes' it is the y vectors of class labels
#note, random_state provides specific seed for pseudorandom generator to allow reproducible analysis of the model
#remove this parameter to allow random selection each run
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify=y, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
model = LogisticRegression(C=0.1, penalty=None, solver='saga')

In [ ]:

#following lines are not needed as gridsearchCV does 'refit' (retrains best estimator on full set provided[the whole
#training set in this case]) by default
model.set_params(max_iter=5000)
model.fit(X_train, y_train)

yhat = model.predict(X_test)
print('Predicted Classes:')
print(yhat)
print('Actual Classes:')
print(y_test)

score = model.score(X_test,y_test)
# report the model performance
print('Accuracy: %.3f ' % (score))

In [ ]:
# predict probabilities with a multinomial logistic regression model
from sklearn.datasets import make_classification
# predict a multinomial probability distribution
yprobs = model.predict_proba(X_test)
# summarize the predicted probabilities
print('Predicted Probabilities:')
print(yprobs)

In [ ]:
#following is test code to output the probabilities in an easy to 
#read format using the dataframe display and format options
import pandas as pd
def plot_probabilities(prob_array, col_labels, sample_indices):
    if yprobs.shape[1] == len(classes):
        prob_df = pd.DataFrame(prob_array, columns=col_labels)
        prob_df['original sample index'] = sample_indices
        pd.set_option('display.float_format', lambda x: '%.3f' % x)
        pd.set_option('display.precision', 3)
        display(prob_df)
    else:
        print('Incorrect label list length')
        
classes = ['Negative','Positive']   
rows = y_test.index
print("Probabilities:")
plot_probabilities(yprobs , classes, rows)

In [ ]:
# Compute confusion matrix
from sklearn.metrics import classification_report, multilabel_confusion_matrix, confusion_matrix
cnf_matrix = confusion_matrix(y_test, yhat)
np.set_printoptions(precision=2)

print (classification_report(y_test, yhat))

# Plot non-normalized confusion matrix
#plt.figure()
#plot_confusion_matrix(cnf_matrix, classes=['Negative','Pos'],normalize= False,  title='Confusion matrix')
print(cnf_matrix)